<a href="https://colab.research.google.com/github/daniyal1d/Qdrant/blob/main/Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-generativeai qdrant-client numpy requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 11.9 MB/s eta 0:00:00


In [4]:
import google.generativeai as genai
from qdrant_client import QdrantClient
import google.generativeai as genai
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
import numpy as np
import requests
# STEP 3: Gemini API Key setup

In [5]:
genai.configure(api_key="")
model = genai.GenerativeModel("gemini-2.0-flash")

In [7]:
qdrant_client = QdrantClient(
    url="",
    api_key=""  # Replace with your real API key
)

collection_name = "ChatMemory"

# Create collection if it doesn't exist
if collection_name not in [c.name for c in qdrant_client.get_collections().collections]:
    qdrant_client.recreate_collection(
        collection_name=collection_name,
        vectors_config=rest.VectorParams(size=768, distance=rest.Distance.COSINE),
    )

<ipython-input-7-410c829ed3eb>:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [8]:
def embed(text):
    res = genai.embed_content(
        model="models/embedding-001",
        content=text,
        task_type="RETRIEVAL_DOCUMENT"
    )
    return np.array(res["embedding"], dtype=np.float32)

In [9]:
def add_to_memory(text):
    vector = embed(text)
    # Use text hash or UUID for unique ID in production; here text is used as id for demo
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[
            rest.PointStruct(id=hash(text) % (10**9), vector=vector.tolist(), payload={"text": text})
        ],
    )

In [10]:
def search_memory(query, top_k=3):
    vector = embed(query).tolist()
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=vector,
        limit=top_k,
        with_payload=True,
    )
    return [point.payload["text"] for point in search_result]

In [11]:
def chat(user_input, history=[]):
    context = search_memory(user_input)
    context_str = "\n".join(context)

    prompt = f"""You are a helpful assistant. Use the following context to answer the user's question.

Context:
{context_str}

User: {user_input}
"""

    response = model.generate_content(prompt)
    reply = response.text

    # Store user input and model reply
    add_to_memory(user_input)
    add_to_memory(reply)

    history.append((user_input, reply))
    return reply

In [12]:
print("🤖 Gemini Chatbot with Qdrant memory (type 'exit' to quit)\n")
chat_history = []

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    response = chat(user_input, chat_history)
    print(f"Assistant: {response}\n")

🤖 Gemini Chatbot with Qdrant memory (type 'exit' to quit)

You: hello 


<ipython-input-10-3a3d4bbbfd08>:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(


Assistant: Hello! How can I help you today?


You: what is mcp
Assistant: I am sorry, I cannot answer that question. The context does not provide information about what "mcp" means.


You: model context protocol
Assistant: I am not familiar with an MCP with that definition. Can you provide more context? 

You: what can you do 
Assistant: I can provide information, answer questions, generate text, translate languages, and complete tasks as instructed. I am still under development, but I am learning new things every day.


You: what information you ccan provide me
Assistant: Hello! How can I help you today?


You: exit
Goodbye!
